In [1]:
import pickle
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re
import scipy
import numpy as np
from sklearn import preprocessing

Using TensorFlow backend.


In [6]:
filename = "22-04-14_09-37-50_word2vec_Vect_WRul_Max-Len-Seq-38_Freq-318"
path = "F:/RweetMiner/RweetMinerNew/request_identification_class_DLCorrected/word2vec_vect/" + filename

load_vect = pickle.load(open(path, 'rb'))
# load_vect.texts_to_sequences(df["tweet_text"])

In [13]:
fnameTrainingDataset = "datasets/200_random_sample.csv"
df = pd.read_csv(fnameTrainingDataset)
X = df['tweet-text']
y_binary = df['tweet-class']
le = preprocessing.LabelEncoder()
y_binary = le.fit_transform(y_binary)

classes_labels_binary = le.classes_
sequences = load_vect.texts_to_sequences(X)
X_train_WoR_dtm = pad_sequences(sequences, maxlen=38)
X_train_WoR_Features = list(load_vect.word_index.keys())

In [2]:
fnameTrainingDataset = "datasets/200_random_sample.csv"
df = pd.read_csv(fnameTrainingDataset)
df

,Unnamed: 0,tweetID,tweet-text,tweet-class
0,2000,2.640244e+17,issa issa foundation take donation american re...,request
1,2354,2.658464e+17,pray wonderful donor volunteer today ! could p...,request
2,2413,2.654993e+17,good job abc ! _MENTIOM_ : donated # dayofgivi...,request
3,982,2.654694e+17,_RT_ : help hand crush : velah donate proceeds...,not_excl
4,231,2.652074e+17,_RT_ : benefit concert long island hurricane s...,not_excl
...,...,...,...,...
495,2181,2.645877e+17,donate red cross help victim hurricane sandy .,request
496,1437,2.641801e+17,_RT_ : graffiti artist : art show benefit vict...,request
497,1356,2.633728e+17,shout kellogg 's ( kellogg company ) donate $ ...,not_excl
498,2289,2.654531e+17,support new york _RT_ donate # dayofgiving hel...,request


In [13]:
size = 300
embedding_vector = np.ndarray(shape=(size,))
embedding_vector[:] = 1/size
embedding_vector.shape

(300,)

In [15]:
filename = "22-04-14_09-37-50_word2vec_Vect_WRul_Max-Len-Seq-38_Freq-318"
path = "F:/RweetMiner/RweetMinerNew/request_identification_class_DLCorrected/word2vec_vect/" + filename
load_model = pickle.load(open(path, 'rb'))

In [16]:
def apply_regex(text, regex):
    match_found = (re.search(regex ,text) !=None)
    match_found = int(match_found == True)
    return match_found

def series_to_DataFrame(X_data):
    X_data = X_data.to_frame()
    return X_data

def concat_sparse_matrices_h(data_X_dtm, data_Rules_dtm, features_X, features_Rules):
    combined_features = features_X + features_Rules
    concat_sparse = scipy.sparse.hstack([data_X_dtm, data_Rules_dtm], format='csr')
    return concat_sparse, combined_features

def gen_rules_features(X_data_series): # , X_data_dtm, features_arg):
    '''sparse matrix and series matrices should be converted to dataframe for applying rules and treating
    it as features...
    I wrote two functions i.e.,sparse_matrix_to_DataFrame() and series_DataFrame()
      for changing datatypes'''
    X_data_DF = series_to_DataFrame(X_data_series)
#     X_data_DF = X_data_series
    regexes = [
      re.compile(r'\b(I|we)\b.*\b(am|are|will be)\b.*\b(bringing|giving|helping|raising|donating|auctioning)\b', re.I|re.M),
      re.compile(r'\b(I\'m)\b.*\b(bringing|giving|helping|raising|donating|auctioning)\b', re.I|re.M),
      re.compile(r'\b(we\'re)\b.*\b(bringing|giving|helping|raising|donating|auctioning)\b', re.I|re.M),
      re.compile(r'\b(I|we)\b.*\b(will|would like to)\b.*\b(bring|give|help|raise|donate|auction)\b', re.I|re.M),
      re.compile(r'\b(I|we)\b.*\b(will|would like to)\b.*\b(work|volunteer|assist)\b', re.I|re.M),
      re.compile(r'\b(we\'ll)\b.*\b(bring|give|help|raise|donate|auction)\b', re.I|re.M),
      re.compile(r'\b(I|we)\b.*\b(ready|prepared)\b.*\b(bring|give|help|raise|donate|auction)\b', re.I|re.M),
      re.compile(r'\b(where)\b.*\b(can I|can we)\b.*\b(bring|give|help|raise|donate)\b', re.I|re.M),
      re.compile(r'\b(where)\b.*\b(can I|can we)\b.*\b(work|volunteer|assist)\b', re.I|re.M),
      re.compile(r'\b(I|we)\b.*\b(like|want)\b.*\bto\b.*\b(bring|give|help|raise|donate)\b', re.I|re.M),
      re.compile(r'\b(I|we)\b.*\b(like|want)\b.*\bto\b.*\b(work|volunteer|assist)\b', re.I|re.M),
      re.compile(r'\b(will be)\b.*\b(brought|given|raised|donated|auctioned)\b', re.I|re.M),
      re.compile(r'\b\w*\s*\b\?', re.I|re.M),
      re.compile(r'\b(you|u).*(can|could|should|want to)\b', re.I|re.M),
      re.compile(r'\b(can|could|should).*(you|u)\b', re.I|re.M),
      re.compile(r'\b(like|want)\b.*\bto\b.*\b(bring|give|help|raise|donate)\b', re.I|re.M),
      re.compile(r'\b(how)\b.*\b(can I|can we)\b.*\b(bring|give|help|raise|donate)\b', re.I|re.M),
      re.compile(r'\b(how)\b.*\b(can I|can we)\b.*\b(work|volunteer|assist)\b', re.I|re.M)

    ]
    temp = pd.DataFrame()
    features_arg = []
    for i, regex in zip(range(len(regexes)), regexes):
      columnName = "RegEx_" + str(i + 1)
      features_arg.append(columnName)
      temp[columnName] = X_data_DF['tweet-text'].apply(lambda text: apply_regex(text, regex))
    temp_sparse = scipy.sparse.csr_matrix(temp.values)
    return temp_sparse, features_arg

In [14]:
fnameTrainingDataset = "datasets/200_random_sample.csv"
df = pd.read_csv(fnameTrainingDataset)
X = df['tweet-text']
y_binary = df['tweet-class']
le = preprocessing.LabelEncoder()
y_binary = le.fit_transform(y_binary)

classes_labels_binary = le.classes_
X_train_WoR_dtm = load_vect.transform(X)
X_train_WoR_Features = list(load_vect.get_feature_names_out())

AttributeError: 'Tokenizer' object has no attribute 'transform'

In [173]:
X_Rules_dtm, features_Rules = gen_rules_features(X)
X_train_WR_dtm, combined_features = concat_sparse_matrices_h(X_train_WoR_dtm, X_Rules_dtm, X_train_WoR_Features, features_Rules)

In [174]:

y_bin_pred = load_model.predict(X_train_WR_dtm)

In [175]:
import numpy as np

df['DT'] = y_bin_pred.tolist()
df['GB'] = np.random.randint(2, size=500).tolist()
df['MLP'] = np.random.randint(2, size=500).tolist()
df['RF'] = np.random.randint(2, size=500).tolist()

In [134]:
df["request"] = df.iloc[:, -4:].mean(axis=1) > 0.6

In [176]:
df.iloc[:, -4:]

,DT,GB,MLP,RF
0,0,0,1,0
1,1,0,1,1
2,0,1,1,1
3,0,1,1,1
4,0,1,0,0
...,...,...,...,...
495,1,0,0,1
496,0,0,1,1
497,0,0,1,1
498,0,1,0,1


In [358]:
e = np.empty([0])
e = np.append(e, 1)
e = np.append(e, 2)
e = np.append(e, 3)
print(list(e))

[1.0, 2.0, 3.0]


In [345]:
df.iloc[:, -2:].values

array([[1, 0],
       [2, 2],
       [3, 0],
       [0, 0]], dtype=int64)

In [171]:
df.iloc[:, -:]

,GB,MLP,RF,num_uniq
0,0,1,1,4
1,0,0,0,4
2,0,0,0,4
3,1,1,0,4
4,1,1,1,4
...,...,...,...,...
495,0,0,0,4
496,1,0,1,4
497,0,0,0,4
498,0,0,0,4


In [366]:
def get_max_from_rows_into_column(df, indexes):
    """ The function takes dataframe and index of columns, and 
    then it returns the array containg the most frequent value against each row.
    Example: 
    dataframe:
    'c1' | 'c2'| 'c3' | 'c4'
    -------------------------
      1  |  1  |   1  |  0  
      2  |  1  |   2  |  2
      2  |  3  |   3  |  0 
      2  |  0  |   0  |  0 
      returned array:
      (1, 2, 3, 0)
      df: dataframe
      indexes: indexes of columns
      example: get_max_from_rows_into_column(dataframe, -3:)
      """
    def get_array(df):
        max_unique_label_column = np.empty([0])
        for v in df.values:
            unique_label, freq_label = np.unique(v, return_counts=True)
            print(unique_label[np.argmax(freq_label)])
            max_unique_label_column = np.append(max_unique_label_column, unique_label[np.argmax(freq_label)])
            
#             print(unique_label[np.argmax(freq_label)])
        return max_unique_label_column
    if ":" in indexes:
        if indexes[0] == ":":
            
            df = df.iloc[:, :int(indexes[1:])]
            max_unique_label_column = get_array(df)
        if indexes[-1] == ":":
            df = df.iloc[:, int(indexes[:-1]):]
            max_unique_label_column = get_array(df)
           
        else:
            indexes = [int(x) for x in indexes.split(":")]
            df = df.iloc[:, indexes[0]: indexes[1]]
            max_unique_label_column = get_array(df)
    else:
            indexes = [int(x) for x in indexes.split(",")]
            df = df.iloc[:, indexes]
            max_unique_label_column = get_array(df)
    print(max_unique_label_column)
    return max_unique_label_column
#     for v in df.iloc[:, indexes].values:
#         unique_label, freq_label = np.unique(v, return_counts=True)
#         print(unique_label)
#         print(freq_label)
#         print(unique_label[np.argmax(freq_label)])
#     #     print(counts.indexOf(max(counts)))
    #     print(unique)

#         print(40*"*")

d = {'c1': [1, 2, 2, 2], 'c2': [1, 1, 3, 0], 'c3': [1, 2, 3, 0], 'c4': [0, 2, 0, 0]}
df = pd.DataFrame(data=d)

indexes = "-3:"
df["new"] = get_max_from_rows_into_column(df, indexes)
df

1
2
3
0
[1. 2. 3. 0.]


,c1,c2,c3,c4,new
0,1,1,1,0,1.0
1,2,1,2,2,2.0
2,2,3,3,0,3.0
3,2,0,0,0,0.0


In [300]:
indexes = "-4:-3"
print(indexes)
if ":" in indexes:
    if indexes[0] == ":" or indexes[-1] == ":":
        indexes = indexes
    else:
        indexes = [int(x) for x in indexes.split(":")]
        return
else:
    indexes = [int(x) for x in indexes.split(",")]
print(indexes)
df.iloc[:, indexes]


-4:-3
[-4, -3]


,DT,GB
0,0,0
1,1,0
2,0,1
3,0,1
4,0,1
...,...,...
495,1,0
496,0,0
497,0,0
498,0,1


In [335]:
d = {'c1': [1, 2, 2, 2], 'c2': [1, 1, 3, 0], 'c3': [1, 2, 2, 0], 'c4': [0, 2, 0, 0]}
df = pd.DataFrame(data=d)
df

,c1,c2,c3,c4
0,1,1,1,0
1,2,1,2,0
2,2,3,2,0
3,2,0,0,0


In [19]:
from sklearn.feature_extraction.text import CountVectorizer
fnameTrainingDataset = "datasets/500_random_sample.csv"
df = pd.read_csv(fnameTrainingDataset)
X = df['tweet_text']
Vect = CountVectorizer(ngram_range=(2,2), min_df=1, max_df=1)
Vect.fit(X)
X_train_WoR_dtm = Vect.transform(X)
X_train_WoR_dtm

<500x2600 sparse matrix of type '<class 'numpy.int64'>'
	with 2600 stored elements in Compressed Sparse Row format>